In [23]:
from pyhive import hive

In [25]:
def run_hive_query(query):
    cursor = hive.connect('analytics1003.eqiad.wmnet', 10000).cursor()
    cursor.execute(query)
    return cursor.fetchall()

In [ ]:
def create_slice_table():
    run_hive_query("""
        create table fdans.ua_week_sample2 as
            select user_agent, user_agent_map as after, ua_parser(user_agent) as before, agent_type, day
            from wmf.webrequest
            where webrequest_source='text'
            and year = 2018
            and month = 7
            and array_contains(array(8,9,10,11,12,13,14), day)
            and is_pageview;
    """)

In [28]:
number_of_pageviews = run_hive_query('SELECT count(1) from fdans.ua_week_sample2')[0][0]
print('TOTAL PAGEVIEW COUNT: ', number_of_pageviews)

TOTAL PAGEVIEW COUNT:  4309356985


In [31]:
number_of_spiders_before = run_hive_query("SELECT SUM(views) from fdans.ua_week_maps where before['device_family']='Spider'")

In [ ]:
print('TOTAL SPIDERS BEFORE:', number_of_spiders_before[0][0] )

In [33]:
increment_per_day = run_hive_query('''
with 
bots_before as (
    select day, count(1) views_before
    from fdans.ua_week_sample2
    where before['device_family'] = 'Spider'
    group by day
),
bots_after as (
    select day, count(1) views_after
    from fdans.ua_week_sample2
    where after['device_family'] = 'Spider'
    group by day
)
select bots_before.day, bots_before.views_before, a.views_after, 100*(a.views_after - bots_before.views_before)/bots_before.views_before from bots_before
join bots_after a on (bots_before.day = a.day)
''')

In [36]:
print('Starts Sunday July 8')
for day in increment_per_day:
    print(day[0], day[3])

Starts Sunday July 8
8 0.24473130227252016
9 0.28833015440792586
10 0.30091474582320593
11 0.30980127867718077
12 0.29938768251170145
13 0.2961167522526749
14 0.26241725804885935
